# Project_final

## by Emanuel Thomas & Sweety Saha

Read data.

In [1]:
import pandas as pd
import numpy as np

team_info = pd.read_excel("project_final.xlsx",
                       sheet_name="workers",
                       index_col=0)
seat_info = pd.read_excel("project_final.xlsx",
                       sheet_name="seat_assign",
                       index_col=0)

distance_info = pd.read_excel("project_final.xlsx",
                       sheet_name="distance",
                       index_col=0)

distance=distance_info.stack().to_dict()

c={}

workers=list(team_info.index)
print(workers)

teams=team_info.to_dict()['Team']
print(teams)

seats=team_info.to_dict()['Current Location']
print(seats)

print("ranks")

rank=list(team_info.to_dict()['Rank'])
print(rank)

ranks=list(seat_info.to_dict()['Workspace Rank'])

rk=list(rank + ranks)
print("rank of members")
#print(team_info['Rank'])
print(team_info.to_dict()['Rank'])

 
seats_list=list(seat_info.index)

['Worker 1', 'Worker 2', 'Worker 3', 'Worker 4', 'Worker 5', 'Worker 6', 'Worker 7', 'Worker 8', 'Worker 9', 'Worker 10', 'Worker 11', 'Worker 12', 'Worker 13', 'Worker 14', 'Worker 15', 'Worker 16', 'Worker 17', 'Worker 18', 'Worker 19', 'Worker 20']
{'Worker 1': 'Team 1', 'Worker 2': 'Team 1', 'Worker 3': 'Team 1', 'Worker 4': 'Team 1', 'Worker 5': 'Team 2', 'Worker 6': 'Team 2', 'Worker 7': 'Team 2', 'Worker 8': 'Team 2', 'Worker 9': 'Team 2', 'Worker 10': 'Team 2', 'Worker 11': 'Team 3', 'Worker 12': 'Team 3', 'Worker 13': 'Team 3', 'Worker 14': 'Team 3', 'Worker 15': 'Team 3', 'Worker 16': 'Team 3', 'Worker 17': 'Team 3', 'Worker 18': 'Team 3', 'Worker 19': 'Team 3', 'Worker 20': 'Team 3'}
{'Worker 1': 'Workspace 1', 'Worker 2': 'Workspace 4', 'Worker 3': 'Workspace 5', 'Worker 4': 'Workspace 7', 'Worker 5': 'Workspace 3', 'Worker 6': 'Workspace 5', 'Worker 7': 'Workspace 7', 'Worker 8': 'Workspace 4', 'Worker 9': 'Workspace 6', 'Worker 10': 'Workspace 6', 'Worker 11': 'Workspace 

In [2]:
capacity=seat_info['Capacity']
print(capacity)

Workspace
Workspace 1    1
Workspace 2    1
Workspace 3    1
Workspace 4    6
Workspace 5    7
Workspace 6    6
Workspace 7    7
Name: Capacity, dtype: int64


In [3]:
r= dict(zip(workers,[1] * len(workers)))
r.update(dict(zip(rk, [1] * len(seats_list))))
print(r)

{'Worker 1': 1, 'Worker 2': 1, 'Worker 3': 1, 'Worker 4': 1, 'Worker 5': 1, 'Worker 6': 1, 'Worker 7': 1, 'Worker 8': 1, 'Worker 9': 1, 'Worker 10': 1, 'Worker 11': 1, 'Worker 12': 1, 'Worker 13': 1, 'Worker 14': 1, 'Worker 15': 1, 'Worker 16': 1, 'Worker 17': 1, 'Worker 18': 1, 'Worker 19': 1, 'Worker 20': 1}


In [4]:
# cost of moving from one seat to other is 1 and 0 otherwise

for w,s in seats.items():
    for d in seats_list:
        if s == d:
            c[(w,d)]=0
        else:
            c[(w,d)]=1
            
            
print(c)
A = list(c.keys())
print(A)

V = workers + seats_list
print(V)

b= dict(zip(workers,[1] * len(workers)))
#b.update(dict(zip(seats_list, -capacity * len(seats_list))))
b.update(dict(zip(seats_list, -capacity)))
print(b)


{('Worker 1', 'Workspace 1'): 0, ('Worker 1', 'Workspace 2'): 1, ('Worker 1', 'Workspace 3'): 1, ('Worker 1', 'Workspace 4'): 1, ('Worker 1', 'Workspace 5'): 1, ('Worker 1', 'Workspace 6'): 1, ('Worker 1', 'Workspace 7'): 1, ('Worker 2', 'Workspace 1'): 1, ('Worker 2', 'Workspace 2'): 1, ('Worker 2', 'Workspace 3'): 1, ('Worker 2', 'Workspace 4'): 0, ('Worker 2', 'Workspace 5'): 1, ('Worker 2', 'Workspace 6'): 1, ('Worker 2', 'Workspace 7'): 1, ('Worker 3', 'Workspace 1'): 1, ('Worker 3', 'Workspace 2'): 1, ('Worker 3', 'Workspace 3'): 1, ('Worker 3', 'Workspace 4'): 1, ('Worker 3', 'Workspace 5'): 0, ('Worker 3', 'Workspace 6'): 1, ('Worker 3', 'Workspace 7'): 1, ('Worker 4', 'Workspace 1'): 1, ('Worker 4', 'Workspace 2'): 1, ('Worker 4', 'Workspace 3'): 1, ('Worker 4', 'Workspace 4'): 1, ('Worker 4', 'Workspace 5'): 1, ('Worker 4', 'Workspace 6'): 1, ('Worker 4', 'Workspace 7'): 0, ('Worker 5', 'Workspace 1'): 1, ('Worker 5', 'Workspace 2'): 1, ('Worker 5', 'Workspace 3'): 0, ('Worke

Import Pyomo and define model.

In [5]:
from pyomo.environ import *
model = ConcreteModel()

In [6]:
try:
    model.del_component(model.x)
    model.del_component(model.x_index)
    model.del_component(model.y)
    model.del_component(model.y_index)
except:
    print("Defining variables for the first time")
model.x = Var(A, domain=Binary)
model.y=  Var(A,A, domain=Binary)

Defining variables for the first time


Define objective function.

In [7]:
try:
    model.del_component(model.Cost)
except:
    print("Defining objective for the first time")
model.Cost = Objective(expr=sum(c[i,j]*model.x[i,j] for (i,j) in A)+
                       sum(sum(distance[j,l]*model.y[i,j,k,l] for (i,j) in A) for (k,l) in A),
                       sense=minimize)

Defining objective for the first time


Define the constraints

In [8]:
try:
    model.del_component(model.FlowBalance)
    model.del_component(model.FlowBalance_index)
except:
    print("Defining constraints for the first time")
    
model.FlowBalance = ConstraintList()
for i in V:
    if i in workers:
        model.FlowBalance.add(sum(model.x[i,j] for j in V if (i,j) in A) - sum(model.x[j,i] for j in V if (j,i) in A) == b[i])
    else:
        model.FlowBalance.add(sum(model.x[i,j] for j in V if (i,j) in A) - sum(model.x[j,i] for j in V if (j,i) in A) >= b[i])
#model.pprint()



Defining constraints for the first time


Specify distance constraint

In [9]:
try:
    model.del_component(model.DistanceConstraint)
    model.del_component(model.DistanceConstraint_index)
except:
    print("Defining Distance constraint for the first time")

model.DistanceConstraint = ConstraintList()

for (i,j) in A:
    for(k,l) in A:
        if teams[i]==teams[k]:
            model.DistanceConstraint.add(model.x[i,j] + model.x[k,l] <= model.y[i,j,k,l]+1)


Defining Distance constraint for the first time


In [10]:
try:
    model.del_component(model.RankConstraint)
    model.del_component(model.RankConstraint_index)
except:
    print("Defining RankConstraint constraint for the first time")


model.RankConstraint = ConstraintList()
for (i,j) in A:
    if r[i]!=r[k]:
        model.RankConstraint.add(model.x[i,j] ==0)


Defining RankConstraint constraint for the first time


Initialize solver and get status

In [11]:
solver = SolverFactory('glpk')
solver.options['tmlim'] = 100
solver.options['memlim'] = 100
status = solver.solve(model, tee=True)
print('Status = %s' % status)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 100 --memlim 100 --write C:\Users\pinga\AppData\Local\Temp\tmpfsccmoc0.glpk.raw
 --wglp C:\Users\pinga\AppData\Local\Temp\tmpwl_dzotx.glpk.glp --cpxlp C:\Users\pinga\AppData\Local\Temp\tmpe302fzbw.pyomo.lp
Reading problem data from 'C:\Users\pinga\AppData\Local\Temp\tmpe302fzbw.pyomo.lp'...
C:\Users\pinga\AppData\Local\Temp\tmpe302fzbw.pyomo.lp:79846: warning: lower bound of variable 'x1' redefined
C:\Users\pinga\AppData\Local\Temp\tmpe302fzbw.pyomo.lp:79846: warning: upper bound of variable 'x1' redefined
7476 rows, 18005 columns, 22485 non-zeros
18004 integer variables, all of which are binary
97850 lines were read
Writing problem data to 'C:\Users\pinga\AppData\Local\Temp\tmpwl_dzotx.glpk.glp'...
72366 lines were written
GLPK Integer Optimizer, v4.65
7476 rows, 18005 columns, 22485 non-zeros
18004 integer variables, all of which are binary
Preprocessing...
143 hidden packing inequaliti(es) were de

In [12]:
for (i,j) in A:
    print("%s = %f" % (model.x[i,j], value(model.x[i,j])))
print("Cost = %f" % value(model.Cost))

x[Worker 1,Workspace 1] = 0.000000
x[Worker 1,Workspace 2] = 0.000000
x[Worker 1,Workspace 3] = 0.000000
x[Worker 1,Workspace 4] = 0.000000
x[Worker 1,Workspace 5] = 1.000000
x[Worker 1,Workspace 6] = 0.000000
x[Worker 1,Workspace 7] = 0.000000
x[Worker 2,Workspace 1] = 0.000000
x[Worker 2,Workspace 2] = 0.000000
x[Worker 2,Workspace 3] = 0.000000
x[Worker 2,Workspace 4] = 0.000000
x[Worker 2,Workspace 5] = 1.000000
x[Worker 2,Workspace 6] = 0.000000
x[Worker 2,Workspace 7] = 0.000000
x[Worker 3,Workspace 1] = 0.000000
x[Worker 3,Workspace 2] = 0.000000
x[Worker 3,Workspace 3] = 0.000000
x[Worker 3,Workspace 4] = 0.000000
x[Worker 3,Workspace 5] = 1.000000
x[Worker 3,Workspace 6] = 0.000000
x[Worker 3,Workspace 7] = 0.000000
x[Worker 4,Workspace 1] = 0.000000
x[Worker 4,Workspace 2] = 0.000000
x[Worker 4,Workspace 3] = 0.000000
x[Worker 4,Workspace 4] = 0.000000
x[Worker 4,Workspace 5] = 1.000000
x[Worker 4,Workspace 6] = 0.000000
x[Worker 4,Workspace 7] = 0.000000
x[Worker 5,Workspace